In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

/home/boada/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/boada/.local/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Load data

In [4]:
inspections = pd.read_csv('./data/inspections.csv')
nypd = pd.read_csv('./data/nypd.csv')
threeoneone = pd.read_csv('./data/311.csv')